In [1]:
# import matplotlib.pyplot as plt
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

In [2]:
class Dataset(torch.utils.data.Dataset):
    """
    overload dataset
    """

    def __init__ (self, x, y):
        self.x = x
        self.y = y
        # enforce to use of float32 or some models will complain
        self.x = self.x.float()
        self.y = self.y.float()

    def __len__ (self): 
        return(len(self.y))

    def __getitem__ (self, idx):
        return self.x[idx], self.y[idx]

In [3]:
# load the data
data = torch.load('data/brain_cancer_dataset_small.pt')

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [5]:
class BrainClassifier(nn.Module):
    def __init__(self):
        super(BrainClassifier, self).__init__()

        # Define layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()

        # Calculate the input size of the fully connected layer according to the output of the conv layers
        fc_input_size = 64 * 64 * 64
        self.fc = nn.Linear(fc_input_size, 3) # 3 classes

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)

        x = F.relu(self.conv2(x))
        x = self.pool2(x)

        x = F.relu(self.conv3(x))
        x = self.pool3(x)

        x = self.flatten(x)

        x = self.fc(x)
        # by default CrossEntropyLoss compute the softmax itself.
        # if leaved uncommented somewhy (????) the model does not learn 
        # remeber to put by hand when you do predictions
        # x = F.softmax(x, dim=1)  

        return x


In [6]:
class minst_classifier(nn.Module):
    def __init__(self):
        super(minst_classifier, self).__init__()

        # Define layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Calculate the flattened size
        self.flatten_size = 64 * 3 * 3  # Adjusted from 4*4 to 3*3
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(self.flatten_size, 10)  # Adjust accordingly

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)

        x = F.relu(self.conv2(x))
        x = self.pool2(x)

        x = F.relu(self.conv3(x))
        x = self.pool3(x)

        x = self.flatten(x)
        x = self.fc(x)
        x = F.softmax(x, dim=1)

        return x

In [7]:
# Define the training loop
def train(model, train_data, val_data, epochs=10, lr=0.001):
    # Define the loss function
    criterion = nn.CrossEntropyLoss()
    # Define the optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Store the losses
    train_losses = []
    val_losses = []

    # Move model to the appropriate device
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(epochs):
        # Set the model to training mode
        model.train()

        # Iterate over the training data
        for i, (img, label) in enumerate(train_data):
            # Move data to the appropriate device
            img, label = img.to(device), label.to(device)

            # Zero the gradients
            optimizer.zero_grad()
            # Forward pass
            y_pred = model(img)
            # Compute the loss
            loss = criterion(y_pred, label.long() - 1)
            # loss = criterion(y_pred, label.long())
            # Backward pass
            loss.backward()
            # Update the weights
            optimizer.step()
            # Store the loss
            train_losses.append(loss.item())
            # Print the loss
            # if i % 10 == 0:
            #     print(f'Epoch {epoch}, Iteration {i}, Loss {loss.item()}')

        # Set the model to evaluation mode
        model.eval()
        # Compute the validation loss
        val_loss = 0
        with torch.no_grad():
            for img, label in val_data:
                # Move data to the appropriate device
                img, label = img.to(device), label.to(device)
                y_pred = model(img)
                loss = criterion(y_pred, label.long() - 1)
                # loss = criterion(y_pred, label.long())
                val_loss += loss.item()

        val_losses.append(val_loss / len(val_data))

        print(f'Epoch {epoch}, Validation Loss {val_loss / len(val_data)}')

    return train_losses, val_losses

In [8]:
# Define the transformations
# transform = transforms.Compose([
#     transforms.ToTensor(),  # Converts to float32 and scales to [0, 1]
#     transforms.Normalize((0.5,), (0.5,))  # Normalize with mean 0.5 and std 0.5 (example values, adjust if needed)
# ])

# Load the dataset
# train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
# val_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# # Create data loaders
train_loader = DataLoader(data, batch_size=128, shuffle=True)
val_loader = DataLoader(data, batch_size=128, shuffle=False)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [9]:
# Example instantiation and training
model = BrainClassifier()
# model = minst_classifier()
train_losses, val_losses = train(model, train_loader, val_loader, epochs=10, lr=0.001)

Epoch 0, Validation Loss 517.2254943847656
Epoch 1, Validation Loss 17.13946835199992
Epoch 2, Validation Loss 6.311733945611347
Epoch 3, Validation Loss 0.9515390495459238
Epoch 4, Validation Loss 0.44365549087524414
Epoch 5, Validation Loss 0.7678773974378904
Epoch 6, Validation Loss 0.04298717404405276
Epoch 7, Validation Loss 0.026183693689138938
Epoch 8, Validation Loss 0.020321732852607965
Epoch 9, Validation Loss 0.016199760449429352


In [10]:
# plot the losses
plt.plot(val_losses, label='Validation Loss')
plt.plot(train_losses, label='Training Loss')

plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.legend()
plt.xlim(0, 200)
plt.show()

NameError: name 'plt' is not defined

In [ ]:
# giusto pre vedere che cos'è il dataset usato

In [ ]:
type(train_dataset)
train_dataset[0]

In [ ]:
# get the dtype of the data
train_dataset[0][0].dtype